In [24]:
# UserInput.py
# Given new user input, recommends a new beer
# N is number of reviews
# if N < 10, recommend based on content
# if N >= 10, recommend based on collaborative filtering

In [175]:
import numpy as np
import pandas as pd
import cPickle as pickle
import time

In [176]:
# load matrix here
dummy = np.load('ratings_mat.npy')
# with open( , 'rb') as f:
#    user_items = pickle.load(f)

# insert user info here
# new_user = np.repeat(np.nan, user_items.shape[1])
# insert ratings from user input to fill in rated beers

# load beer dictionary here
with open('beer_dict.p', 'rb') as f:
    beer_names = pickle.load(f)

In [177]:
print dummy.shape
homer = np.repeat(np.nan, dummy.shape[1])
for i in np.random.randint(0, dummy.shape[1], 10):
    homer[i] = np.random.randint(1,6,1)[0]
homer

(16926, 1269)


array([ nan,  nan,  nan, ...,  nan,  nan,  nan])

## Content Based

In [178]:
# INSERT HERE

 ## CF

In [179]:
# dummy matrix and user input
#dummy = np.matrix(np.random.randint(1,6, (15,15)))
#homer = np.array([5,np.nan,1,2,3,5,5,5,3,np.nan,1,2,3,5,5])
#print dummy, homer

In [180]:
index = np.where(~np.isnan(homer))[0]
missing_index = np.where(np.isnan(homer))[0]
# take items as columns
dummy_red = dummy[:,index]
dummy_miss = dummy[:, missing_index]
homer2 = homer[index]


print index, missing_index, dummy_red, dummy_miss, homer2

[  67  150  361  645  754  786  789  881 1104 1203] [   0    1    2 ..., 1266 1267 1268] [[ 14.05754986  14.7538352   14.30936108 ...,  15.03618088  16.31951942
   14.36883917]
 [ 14.10982828  15.03974667  14.54934616 ...,  15.30307789  16.49207802
   14.59940825]
 [ 15.67037001  16.3264967   15.49775658 ...,  16.34307432  17.54444523
   15.37419949]
 ..., 
 [ 14.00210724  15.22362652  14.57287388 ...,  15.24411373  16.13290672
   14.66198082]
 [ 14.05037718  15.93011169  14.9567275  ...,  15.65309067  16.55825481
   15.5677721 ]
 [ 15.44491749  16.22988729  15.24766703 ...,  16.1084589   16.90242214
   15.51710434]] [[ 15.37260609  15.6599721   15.23829718 ...,  16.30479145  16.71478615
   16.71687088]
 [ 15.55795612  16.03480635  15.48153067 ...,  16.47781582  16.83671074
   16.92485199]
 [ 16.48925858  17.11251602  16.16220579 ...,  17.75134525  18.48328514
   18.26525536]
 ..., 
 [ 15.92637739  16.32348213  15.70524702 ...,  16.01468134  15.91480083
   16.44088631]
 [ 16.59626884  

In [181]:
# calculate global rating avg mu 
mu = dummy.mean()
print mu

15.8225959262


In [182]:
# normalize via global avg mu
homer3 = (homer2 / (homer2.mean() / mu ))
print homer3
homer3.mean()

[ 19.77824491  15.82259593  19.77824491  15.82259593  19.77824491
   7.91129796   7.91129796  19.77824491  11.86694694  19.77824491]


15.822595926208598

In [183]:
# compute euclidean distance and cosine similarity
start = time.time()
from sklearn.metrics import pairwise as pw
pw_dist = pw.pairwise_distances(dummy_red, homer3).flatten()
print time.time() - start
pw_cos = pw.cosine_similarity(dummy_red,homer3).flatten()
print pw_dist, pw_cos
print time.time() - start

0.00339794158936
[ 14.68910657  14.68467616  14.10519437 ...,  14.55433472  14.05121229
  13.8279497 ] [ 0.9615586   0.960769    0.96286574 ...,  0.96176879  0.96355487
  0.96427007]
0.00968194007874


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [184]:
#from sklearn.neighbors import DistanceMetric
#k = DistanceMetric.get_metric('euclidean')
#k.pairwise(dummy_red, homer3)

In [185]:
# smallest distance are nearest neighbors
pw_dist_df =pd.DataFrame([pw_dist]).transpose()
dist_topn = pw_dist_df.nsmallest(5, 0)
dist_topn

,0
11809,12.468161
16144,12.493434
15430,12.610295
4563,12.683999
3444,12.686825


In [186]:
# largest values are most similar users
pw_cos_df = pd.DataFrame([pw_cos]).transpose()
cos_topn = pw_cos_df.nlargest(5,0)
cos_topn

,0
4563,0.973305
16144,0.971255
8115,0.971096
11809,0.971065
13204,0.970768


In [187]:
# turn distance/similarity into weights
# might need to inspect distance again, right now I simply reversed the weights
dist_weights = np.flipud(np.matrix(dist_topn / sum(dist_topn[0])))

cos_weights = np.matrix(cos_topn / sum(cos_topn[0]))
print dist_weights
print cos_weights

[[ 0.20156145]
 [ 0.20151655]
 [ 0.20034558]
 [ 0.19848897]
 [ 0.19808744]]
[[ 0.20037206]
 [ 0.19995003]
 [ 0.19991728]
 [ 0.19991096]
 [ 0.19984966]]


In [188]:
dummy_miss

array([[ 15.37260609,  15.6599721 ,  15.23829718, ...,  16.30479145,
         16.71478615,  16.71687088],
       [ 15.55795612,  16.03480635,  15.48153067, ...,  16.47781582,
         16.83671074,  16.92485199],
       [ 16.48925858,  17.11251602,  16.16220579, ...,  17.75134525,
         18.48328514,  18.26525536],
       ..., 
       [ 15.92637739,  16.32348213,  15.70524702, ...,  16.01468134,
         15.91480083,  16.44088631],
       [ 16.59626884,  17.25162354,  16.39886351, ...,  16.41061649,
         15.97781439,  16.7520215 ],
       [ 16.87665415,  17.4457119 ,  16.45901527, ...,  16.57495575,
         16.26750345,  17.08984766]])

In [189]:
# get ratings from the top n users for the missing beers
dist_miss_ratings = dummy_miss[dist_topn.index,:] 
cos_miss_ratings = dummy_miss[cos_topn.index,:] 
print dist_miss_ratings
print cos_miss_ratings

[[ 18.03187046  19.1944706   16.33394384 ...,  16.8710825   16.2424118
   17.22907906]
 [ 17.3780899   19.09838562  16.07529955 ...,  16.59189053  16.5840642
   16.9803546 ]
 [ 18.25953884  18.81517735  17.32714958 ...,  16.77321262  16.22618971
   17.19515266]
 [ 16.03840551  19.32007961  14.87852783 ...,  16.75601702  18.43162575
   16.94824724]
 [ 17.2132349   18.69879313  15.93800268 ...,  16.67707644  16.11894379
   17.56938324]]
[[ 16.03840551  19.32007961  14.87852783 ...,  16.75601702  18.43162575
   16.94824724]
 [ 17.3780899   19.09838562  16.07529955 ...,  16.59189053  16.5840642
   16.9803546 ]
 [ 15.43534193  18.19866734  14.30448656 ...,  16.39502039  17.30692873
   16.92717133]
 [ 18.03187046  19.1944706   16.33394384 ...,  16.8710825   16.2424118
   17.22907906]
 [ 17.07784204  18.532654    15.63423262 ...,  16.34009438  16.6283515
   16.67567225]]


In [190]:
# weigh ratings
dist_new_ratings = dist_miss_ratings.transpose() * dist_weights
cos_new_ratings = cos_miss_ratings.transpose() * cos_weights
print dist_new_ratings
print cos_new_ratings

[[ 17.38789298]
 [ 19.02586268]
 [ 16.11349212]
 ..., 
 [ 16.7339435 ]
 [ 16.71808775]
 [ 17.18382793]]
[[ 16.79195915]
 [ 18.86908483]
 [ 15.44504257]
 ..., 
 [ 16.59091131]
 [ 17.03932777]
 [ 16.95212101]]


In [191]:
# find top rating(s)
dist_new_index = np.where(dist_new_ratings == dist_new_ratings.max())[0][0]
cos_new_index = np.where(cos_new_ratings == cos_new_ratings.max())[0][0]
cos_new_index

683

In [192]:

beer_ind_dist =  missing_index[dist_new_index]
beer_ind_cos = missing_index[cos_new_index]

# match index with beer name

In [193]:
print beer_names[beer_ind_dist]
print beer_names[beer_ind_cos]

Alchemist Heady Topper
Proclamation Derivative Citra Hopped Pale Ale
